<a href="https://colab.research.google.com/github/aliciawill/BigData/blob/master/%EC%98%81%ED%99%94%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_surprise%EC%9D%B4%EC%9A%A9-%EC%88%98%EC%97%85%EC%A4%91%EC%83%9D%EC%84%B1-4-%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C-%ED%92%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
#!pip install scikit-surprise

In [58]:
import surprise

In [59]:
print(surprise.__version__)

1.1.1


In [60]:
from surprise import SVD 
## 고유값분해(행렬을 더 낮은 차원으로 분해)해서 다시 원본으로 살리면서 잠재적인 값을 추론하는 방법
from surprise import Dataset, Reader ## SVD를 사용하기 위한 데이터셋 만들어주는 클래스
                            ## 사용자, 아이템, 평점
from surprise import accuracy ## RMSE, MAE, cross-validation(k-fold)
from surprise.model_selection import train_test_split ## 훈련/검증 데이터 분류

In [61]:
import pandas as pd

In [62]:
ratings = pd.read_csv('movie_rating.csv')
ratings

,critic,title,rating
0,Jack,Lady,3.0
1,Jack,Snakes,4.0
2,Jack,You Me,3.5
3,Jack,Superman,5.0
4,Jack,The Night,3.0
5,Mick,Lady,3.0
6,Mick,Snakes,4.0
7,Mick,Just My,2.0
8,Mick,Superman,3.0
9,Mick,You Me,2.0


In [63]:
movies2 = set(ratings['title'])
print(movies2)
print(len(movies2))

{'Just My', 'The Night', 'Superman', 'Lady', 'You Me', 'Snakes'}
6


In [64]:
reader = Reader(rating_scale=(0.5, 5.0))

In [65]:
## 데이터 분류, 객체생성, 훈련시키고, 검증해보고, 검증결과 프린트

In [66]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='movie_rating2.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [67]:
algo = SVD(n_factors=20, n_epochs= 30,  random_state=1)
algo.fit(trainset) 
#predictions = algo.test( testset )
#accuracy.rmse(predictions)

In [68]:
from surprise.model_selection import cross_validate

In [69]:
movies = pd.read_csv('movie_rating.csv')
set(movies['title'])

{'Just My', 'Lady', 'Snakes', 'Superman', 'The Night', 'You Me'}

In [70]:
uid = str('Toby')
iid = str('Lady')

pred = algo.predict(uid, iid, verbose=True)

user: Toby       item: Lady       r_ui = None   est = 3.06   {'was_impossible': False}


In [71]:
def get_unseen_surprise2(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['critic']== userId]['title'].tolist()
    print('본 영화수 >> ', len(seen_movies))

    # 모든 영화들의 movieId를 리스트로 생성. 
    print('전체 영화수 >> ', len(movies))

    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(movies))
    
    return unseen_movies

unseen_movies2 = get_unseen_surprise2(ratings, movies2, 'Toby')

본 영화수 >>  3
전체 영화수 >>  6
평점 매긴 영화수: 3 추천대상 영화수: 3 전체 영화수: 6


In [72]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=2):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(one):
        return one.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [pred.iid for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_preds = list(zip(top_movie_ids, top_movie_rating))
    
    return top_movie_preds


In [73]:
top_predictions = recomm_movie_by_surprise(algo, 'Toby', unseen_movies2)
top_predictions

[('The Night', 3.256663666552331), ('Lady', 3.0646408299625856)]